<h1 style="font-size:42px; text-align:center; margin-bottom:30px;">SPARK ASSIGNMENT</h1>

<h2 style="font-size:21px; text-align:center; margin-bottom:30px;">Submitted by Shubham bansla (18037)</h2>


<br><hr style="border-color:black;background-color:royalblue;height:1px;">

## <span style="color:RoyalBlue">Importing Packages and libraries </span>

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import StringIndexer
from numpy import array

In [2]:
sc = SparkContext.getOrCreate()

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

<br><hr style="border-color:black;background-color:royalblue;height:1px;">

## <span style="color:RoyalBlue">Question NO. 1</span>
#### Write a Pyspark code for decision tree like exercise you have completed in pyhton.

In [4]:
# Loading the data set
data = spark.read.csv(r"C:\Users\shubh\Desktop\SPARK\iris1.csv",header=True)

In [5]:
data.show()

+------------+-----------+------------+-----------+-------+
|Sepal.Length|Sepal.Width|Petal.Length|Petal.Width|Species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|          3|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|           5|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         4.6|        3.4|         1.4|        0.3| setosa|
|           5|        3.4|         1.5|        0.2| setosa|
|         4.4|        2.9|         1.4|        0.2| setosa|
|         4.9|        3.1|         1.5|        0.1| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         4.8|        3.4|         1.6|        0.2| setosa|
|         4.8|          3|         1.4|        0.1| setosa|
|         4.3|          3|         1.1| 

In [6]:
# Checking the schema of the data
data.printSchema()

root
 |-- Sepal.Length: string (nullable = true)
 |-- Sepal.Width: string (nullable = true)
 |-- Petal.Length: string (nullable = true)
 |-- Petal.Width: string (nullable = true)
 |-- Species: string (nullable = true)



In [7]:
# Converting Categorical variable into numerical variable
indexer = StringIndexer(inputCol="Species", outputCol="Species_Num")
indexed = indexer.fit(data).transform(data)
data = indexed.drop("Species")

In [8]:
# Splitind data into test and train
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [9]:
def labeledpoint(fields):
    SepalLength = float(fields[0])
    SepalWidth = float(fields[1])
    PetalLength = float(fields[2])
    PetalWidth = float(fields[3])
    Species_Num = int(fields[4])
    
    return LabeledPoint(Species_Num, array([SepalLength, SepalWidth, PetalLength, PetalWidth]))

In [10]:
train = trainingData.rdd

In [11]:
train = train.map(labeledpoint)
#train = train.collect()

In [12]:
# Building decision tree model
model = DecisionTree.trainClassifier(train, numClasses=3, categoricalFeaturesInfo={},
                                    impurity='gini', maxDepth=5, maxBins=10)

In [18]:
test = testData.rdd

In [19]:
test = test.map(labeledpoint)

In [20]:
# Evaluate model on test instances and compute test error
predictions = model.predict(test.map(lambda x: x.features))
labelsAndPredictions = test.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

Test Error = 0.06976744186046512
Learned classification tree model:
DecisionTreeModel classifier of depth 5 with 19 nodes
  If (feature 3 <= 0.4)
   Predict: 2.0
  Else (feature 3 > 0.4)
   If (feature 3 <= 1.7)
    If (feature 2 <= 4.9)
     If (feature 0 <= 5.0)
      If (feature 1 <= 2.4)
       Predict: 0.0
      Else (feature 1 > 2.4)
       Predict: 1.0
     Else (feature 0 > 5.0)
      Predict: 0.0
    Else (feature 2 > 4.9)
     If (feature 0 <= 6.0)
      If (feature 1 <= 2.4)
       Predict: 1.0
      Else (feature 1 > 2.4)
       Predict: 0.0
     Else (feature 0 > 6.0)
      Predict: 1.0
   Else (feature 3 > 1.7)
    If (feature 2 <= 4.9)
     If (feature 1 <= 3.0)
      Predict: 1.0
     Else (feature 1 > 3.0)
      Predict: 0.0
    Else (feature 2 > 4.9)
     Predict: 1.0



## <span style="color:RoyalBlue">Question NO. 2</span>
#### Create array of names and publish name and length of each name.

In [147]:
words =sc.parallelize (["Praxis","Business","School","Post","Graduate"])
length_of_words = words.map(lambda length :(len(length),length))
length_of_words.collect()

[(6, 'Praxis'), (8, 'Business'), (6, 'School'), (4, 'Post'), (8, 'Graduate')]

## <span style="color:RoyalBlue">Question NO. 3</span>
#### Create array of names and find max, min, standard deviation, variance of length of each element in array.

In [159]:
words =sc.parallelize (["Praxis","Business","School","Post","Graduate"])
length_of_words = words.map(lambda length :(len(length)))
def largerThan(x,y):
    if len(x)> len(y): return(x)
    elif len(x)< len(y): return (y)
    else:
        if x>y : return (x)
        else: return(y)
print("Maximum length Word",words.reduce(largerThan))
print("Mininmum Length Word",words.reduce(lambda x,y : x if len(x)< len(y) else y))

print("Maximum length",length_of_words.max())
print("Minimum length",length_of_words.min())
print("Standard deviation",length_of_words.stdev())
print("Variance",length_of_words.variance())





Maximum length Word Graduate
Mininmum Length Word Post
Maximum length 8
Minimum length 4
Standard deviation 1.49666295471
Variance 2.2399999999999998


## <span style="color:RoyalBlue">Question NO. 4</span>
#### Use data set “ocp_data.tsv” attached in sheet (data is delimited with tab space)

###### a.	List no. of shows per year.

In [195]:
# Importing the file
read_file = sc.textFile(r"C:\Users\shubh\Desktop\SPARK\OCP_data.tsv")

# Spliting the data into row and columns
split_file = read_file.map(lambda line: line.split("\t"))

# Taking only two columns that is year and shows

shows_per_year= split_file.map(lambda line :(line[0])) # taking only two columns
shows_per_year.countByValue()

defaultdict(int,
            {'1999': 166,
             '2000': 169,
             '2001': 157,
             '2002': 159,
             '2003': 166,
             '2004': 164,
             '2005': 162,
             '2006': 161,
             '2007': 141,
             '2008': 164,
             '2009': 163,
             '2010': 165,
             '2011': 163,
             '2012': 164,
             '2013': 166,
             '2014': 163,
             '2015': 100,
             'YEAR': 1})

###### b.	List no. of shows per year for every occupation/

In [199]:
shows_per_year_occupation = split_file.map(lambda line :(line[0],line[1]))
shows_per_year_occupation.countByValue()

defaultdict(int,
            {('YEAR', 'GoogleKnowlege_Occupation'): 1,
             ('1999', 'actor'): 53,
             ('1999', 'Comedian'): 5,
             ('1999', 'television actress'): 5,
             ('1999', 'film actress'): 3,
             ('1999', 'Singer-lyricist'): 1,
             ('1999', 'model'): 1,
             ('1999', 'stand-up comedian'): 4,
             ('1999', 'actress'): 42,
             ('1999', 'comedian'): 11,
             ('1999', 'Singer-songwriter'): 2,
             ('1999', 'television personality'): 1,
             ('1999', 'Comic'): 1,
             ('1999', 'rock band'): 2,
             ('1999', 'musician'): 2,
             ('1999', 'Film actor'): 2,
             ('1999', 'Model'): 1,
             ('1999', 'journalist'): 1,
             ('1999', 'NA'): 3,
             ('1999', 'singer-songwriter'): 2,
             ('1999', 'us senator'): 2,
             ('1999', 'film actor'): 2,
             ('1999', 'pianist'): 1,
             ('1999', 'Vocalist'): 2,


###### c.	Find top occupation for every year.

In [390]:
from collections import Counter

In [393]:
xyz =split_file.map(lambda x: (x[0],x[1])).groupByKey().mapValues(list).map(lambda x: (x[0],Counter(x[1]).most_common(1)))
xyz.sortByKey().collect()

[('1999', [('actor', 53)]),
 ('2000', [('actor', 61)]),
 ('2001', [('actor', 62)]),
 ('2002', [('actor', 64)]),
 ('2003', [('actor', 47)]),
 ('2004', [('actor', 31)]),
 ('2005', [('actor', 25)]),
 ('2006', [('actor', 33)]),
 ('2007', [('actor', 15)]),
 ('2008', [('actor', 20)]),
 ('2009', [('actor', 19)]),
 ('2010', [('actor', 30)]),
 ('2011', [('actor', 29)]),
 ('2012', [('actor', 20)]),
 ('2013', [('actor', 37)]),
 ('2014', [('actor', 31)]),
 ('2015', [('actor', 19)]),
 ('YEAR', [('GoogleKnowlege_Occupation', 1)])]

###### d.	Which guest have attended maximum shows

In [349]:
s=split_file.map(lambda line : (line[4],1)).reduceByKey(lambda x,y : x+y)
s.sortBy(ascending=False , keyfunc=lambda s: s[1]).first()

('Fareed Zakaria', 19)

###### e.	List occupation under each group 

In [345]:
split_file.map(lambda line : (line[3],line[1])).collect()

[('Group', 'GoogleKnowlege_Occupation'),
 ('Acting', 'actor'),
 ('Comedy', 'Comedian'),
 ('Acting', 'television actress'),
 ('Acting', 'film actress'),
 ('Acting', 'actor'),
 ('Acting', 'actor'),
 ('Musician', 'Singer-lyricist'),
 ('Media', 'model'),
 ('Acting', 'actor'),
 ('Comedy', 'stand-up comedian'),
 ('Acting', 'actress'),
 ('Acting', 'actor'),
 ('Acting', 'television actress'),
 ('Comedy', 'Comedian'),
 ('Acting', 'actress'),
 ('Acting', 'actress'),
 ('Acting', 'actress'),
 ('Comedy', 'comedian'),
 ('Acting', 'actress'),
 ('Musician', 'Singer-songwriter'),
 ('Acting', 'actor'),
 ('Media', 'television personality'),
 ('Acting', 'actress'),
 ('Comedy', 'Comic'),
 ('Acting', 'actor'),
 ('Acting', 'actress'),
 ('Musician', 'Singer-songwriter'),
 ('Acting', 'actress'),
 ('Comedy', 'comedian'),
 ('Acting', 'actress'),
 ('Musician', 'rock band'),
 ('Musician', 'musician'),
 ('Acting', 'Film actor'),
 ('Media', 'Model'),
 ('Acting', 'television actress'),
 ('Acting', 'actress'),
 ('Acti

###### f.	How many show per group of year and occupation

In [346]:
split_file.map(lambda line : (line[3],line[0],line[1])).countByValue()

defaultdict(int,
            {('Group', 'YEAR', 'GoogleKnowlege_Occupation'): 1,
             ('Acting', '1999', 'actor'): 53,
             ('Comedy', '1999', 'Comedian'): 5,
             ('Acting', '1999', 'television actress'): 5,
             ('Acting', '1999', 'film actress'): 3,
             ('Musician', '1999', 'Singer-lyricist'): 1,
             ('Media', '1999', 'model'): 1,
             ('Comedy', '1999', 'stand-up comedian'): 4,
             ('Acting', '1999', 'actress'): 42,
             ('Comedy', '1999', 'comedian'): 11,
             ('Musician', '1999', 'Singer-songwriter'): 2,
             ('Media', '1999', 'television personality'): 1,
             ('Comedy', '1999', 'Comic'): 1,
             ('Musician', '1999', 'rock band'): 2,
             ('Musician', '1999', 'musician'): 2,
             ('Acting', '1999', 'Film actor'): 2,
             ('Media', '1999', 'Model'): 1,
             ('Media', '1999', 'journalist'): 1,
             ('NA', '1999', 'NA'): 3,
             